In [17]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import vv_plots as vvp
import sys
from PIL import ImageTk, Image
import os

m=tk.Tk()                            # Create instance for master window   
m.geometry("525x650")
m.title("HHC Tools") 
tabControl = tk.ttk.Notebook(m)
font0 = ('arial',8)
font1 = ('arial',10)
font2 = ('arial',12); font2b = ('arial',12,'bold')
font3 = ('arial',14); font3b = ('arial',14,'bold')
font4 = ('arial',16); font4b = ('arial',16,'bold')
font5 = ('arial',19); font5b = ('arial',19,'bold')

#````````````````````````````````````````````````````````#

vv_plot = ttk.Frame(tabControl)

tabControl.add(vv_plot, text='vv_plot')      # Add the tab
tabControl.pack(expand=1, fill="both")  # Pack to make visible

abs_path=sys.path[0]

def vv_ras_file_select_in():
    global vv_ras_plan_file_in, vv_ras_hdf_file_in
    vv_ras_plan_file_in = tk.StringVar().get()
    vv_ras_plan_file_in = tk.filedialog.askopenfilename(initialdir = "/",
                                     title = "Select file",)
    vv_ras_hdf_file_in = vv_ras_plan_file_in+".hdf"

    print(f"RAS plan file is {vv_ras_plan_file_in}")
    print(f"RAS hdf file is {vv_ras_hdf_file_in}")

def vv_CRMS_select_in():
    global vv_CRMS_file_in_0, vv_CRMS_file_in_1
    vv_CRMS_file_in_0 = tk.StringVar().get()
    vv_CRMS_file_in_0 = tk.filedialog.askopenfilename(initialdir = "/",
                                     title = "Select file",filetypes = (("CSV files","*-0.csv"),("all files","*.*")))
    
    vv_CRMS_file_in_1=list(vv_CRMS_file_in_0)
    vv_CRMS_file_in_1[-5]='1'
    vv_CRMS_file_in_1=''.join(vv_CRMS_file_in_1)

    inputs_path_file=abs_path+'\\inputs_path.txt'
    vvp.path_write(inputs_path_file,vv_CRMS_file_in_1)    
    print(f"CRMS0 plan file is {vv_CRMS_file_in_0}")    
    print(f"CRMS1 plan file is {vv_CRMS_file_in_1}")

def vv_projection_select_in():
    global vv_RAS_projection_file
    vv_RAS_projection_file = tk.StringVar().get()
    vv_RAS_projection_file = tk.filedialog.askopenfilename(initialdir = "/",
                                     title = "Select file",filetypes = (("projection","*.prj"),("all files","*.*")))
    prj_path_file=abs_path+'\\prj_path.txt'
    vvp.path_write(prj_path_file,vv_RAS_projection_file)   
    print(f"RAS prj file is {vv_RAS_projection_file}")    
    
vv_ras_plan_file = tk.Button(vv_plot,
                 text="Select RAS plan file",
                 width=19,
                 bg="#101820",
                 fg="#D3BC8D",
                 font=font2b,
                 command=vv_ras_file_select_in)



vv_CRMS_file = tk.Button(vv_plot,
                 text="Select CRMS file",
                 width=19,
                 bg="#101820",
                 fg="#D3BC8D",
                 font=font2b,
                 command=vv_CRMS_select_in)



vv_projection_file = tk.Button(vv_plot,
                 text="Select RAS projection",
                 width=19,
                 bg="#101820",
                 fg="#D3BC8D",
                 font=font2b,
                 command=vv_projection_select_in)



def vv_make_plots():
    vvp.read_ras(vv_ras_hdf_file_in)
    vvp.crms_ras_results(vvp.vv_ras_results,vvp.vv_flow_areas_2d)
    vvp.crms_set_projection()
    vvp.crms_gather_data(vv_CRMS_file_in_0,vv_CRMS_file_in_1)
    vvp.vv_plots(vvp.crms_ras_xy,vvp.crms_ras_swl,vvp.vv_locations,vvp.CRMS_gauge_results,vv_plot_output_dir)

vv_make_plot_button = tk.Button(vv_plot,
                 text="Make plots",
                 width=19,
                 bg='#E38C79',#"#101820",
                 fg="#000000",
                 font=font2b,
                 command=vv_make_plots)

def vv_plot_output_folder():
    global vv_plot_output_dir
    vv_plot_output_dir = tk.StringVar().get()
    vv_plot_output_dir = tk.filedialog.askdirectory(initialdir = os.getcwd(),
                                     title = "Select folder")

vv_plot_output_folder_select = tk.Button(vv_plot,
                 text="Select output location",
                 width=19,
                 bg="#101820",
                 fg="#D3BC8D",
                 font=("arial",12,"bold"),
                 command=vv_plot_output_folder) 



vv_canvas = tk.Canvas(vv_plot, width = 450, height = 300)      
vv_canvas.place(x=50,y=25)

def vv_plot_window():
    global img, img_count
    try:
        img_count+=1
    except:
        img_count=0
  
    img=Image.open(vv_plot_output_dir+"\\vv_"+str(img_count%6)+".png")
    img = img.resize((450, 300),Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    vv_canvas.create_image(200,150,image=img) 
    
vv_plot_window_button = tk.Button(vv_plot,
                 text="Show plots",
                 width=19,
                 bg="#7FE8BE",
                 fg="#000000",
                 font=font2b,
                 command=vv_plot_window)

vv_CRMS_file.place(x=50,y=500)
vv_projection_file.place(x=50,y=450)
vv_make_plot_button.place(x=300,y=450) 
vv_ras_plan_file.place(x=50,y=400)
vv_plot_output_folder_select.place(x=300,y=400)
vv_plot_window_button.place(x=300,y=500) 

m.mainloop()

In [ ]:
import h5py as h5
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

abs_path=sys.path[0]

def path_write(path,string):
    with open(path, 'w') as the_file:
        the_file.write(string)
        the_file.close()

#prj_path_file=abs_path+'\\prj_path.txt'
#inputs_path_file=abs_path+'\\inputs_path.txt'
#string1='C:\\Users\\b2edhjhc\\Desktop\\MVN\\HHC\\ras_tools\\vv_tool\\testing\\prj.prj'
#string2='C:\\Users\\b2edhjhc\\Desktop\\MVN\\HHC\\ras_tools\\vv_tool\\MultiStationChart_Water_Level_2019912142635311-1.csv'
#path_write(path1,string1)
#path_write(path2,string2)        

#vv_plan='D:\East_Atch\EA.p57' #selecting plan file from windows explorer
#vv_hdf=vv_plan+'.hdf'





def crms_set_projection():
    global vv_locations
    print("converting CRMS coordinate to RAS projection")
    os.system('cmd /k "C:\Python27\ArcGIS10.2\python.exe projection.py"')
    vv_locations=np.loadtxt(string2+"//..//test_1.txt",delimiter=',',dtype='str')
    print("done!")
    
def read_ras(hdf):
    global vv_flow_areas_2d, vv_ras_results
    vv_ras_results=h5.File(hdf,'r')
    vv_flow_areas_2d=[]
    for i in range(len(vv_ras_results['/Geometry/2D Flow Areas'].values())):
        if str(list(vv_ras_results['/Geometry/2D Flow Areas'].values())[i]).split()[1] == 'group':
            vv_flow_areas_2d.append(str(list(vv_ras_results['/Geometry/2D Flow Areas'].values())[i])[37:-15])
    return vv_flow_areas_2d
        
def crms_ras_results(results,flow_areas_2d):
    global crms_ras_xy,crms_ras_swl,crms_ras_dep
    xy=np.array([])
    swl=np.array([])
    dep=np.array([])
    for i in range(len(flow_areas_2d)):
        try:
            swl_=results['/Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/2D Flow Areas/'+flow_areas_2d[i]+'/Water Surface']
            xy_=results['/Geometry/2D Flow Areas/'+flow_areas_2d[i]+'/Cells Center Coordinate']
            dep_=results['/Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/2D Flow Areas/'+flow_areas_2d[i]+'/Depth']
        
            xy=np.concatenate([xy,xy_]) if xy.size else xy_
            swl=np.concatenate([swl,swl_],axis=1) if swl.size else swl_
            dep=np.concatenate([dep,dep_],axis=1) if dep.size else dep_
        except:
            pass
    crms_ras_xy=xy
    crms_ras_swl=swl
    crms_ras_dep=dep

def crms_gather_data(CRMS_data,CRMS_info):
    CRMS_data_in=np.loadtxt(CRMS_data,delimiter=',',dtype='str')
    CRMS_info_in=np.loadtxt(CRMS_info,delimiter=',',dtype='str')
    global CRMS_gauge_results
    CRMS_gauge_results=[]
    for i in range(len(CRMS_info_in)-1):
        CRMS_data_temp=[]
        for j in range(len(CRMS_data_in)):
            if CRMS_data_in[j][0]==CRMS_info_in[i+1][0]:
                str2float=CRMS_data_in[j][3].replace('"', '')
                #CRMS_data_temp.append([CRMS_data_in[j][1],float(str2float)])
                CRMS_data_temp.append(float(str2float))
            else:
                pass
        CRMS_gauge_results.append(CRMS_data_temp)    
    
def vv_plots(crms_ras_xy,crms_ras_swl,locations,CRMS):
    global vv_locations_indices
    vv_locations_indices=[]
    for i in range(len(locations[:,0])-1):
        vv_locations_indices.append(griddata((crms_ras_xy[:,0],
                                               crms_ras_xy[:,1]),
                                              np.array(list(range(len(crms_ras_xy)))),
                                              (locations[i+1,0],locations[i+1,1]),
                                              method='nearest'))
        plt.figure()
        plt.ylim(0.0, 5.0) 
        plt.plot(crms_ras_swl[:,vv_locations_indices[i]])
        plt.plot(np.array(CRMS_gauge_results)[i,:])
        plt.savefig('vv_'+str(i)+'.png')
        plt.show()